In [1]:
from scipy.stats import ttest_ind
import pandas as pd
import numpy as np
import re

In [ ]:
file = open('third_model_data/frequent_patterns/translatedFS_q5_high_s5g0', 'r')
FP_high = file.readlines()
file.close()
FP_high_col1 = [x[: x.index(' \t')] for x in FP_high]
FP_high_col2 = [x[x.index('\t')+1:-1] for x in FP_high]
d = {'FP': FP_high_col1, 'FoC_high': FP_high_col2}
df_high = pd.DataFrame(data = d)
df_high

In [ ]:
file = open('third_model_data/frequent_patterns/translatedFS_q5_low_s5g0', 'r')
FP_low = file.readlines()
file.close()
FP_low_col1 = [x[: x.index(' \t')] for x in FP_low]
FP_low_col2 = [x[x.index('\t')+1:-1] for x in FP_low]
d = {'FP': FP_low_col1, 'FoC_low': FP_low_col2}
df_low = pd.DataFrame(data = d)
df_low

In [ ]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [ ]:
df = pd.merge(df_high, df_low, how='outer', on='FP').fillna(0)
df['FoC_high'] = df['FoC_high'].astype(int)
df['FoC_low'] = df['FoC_low'].astype(int)
df

In [ ]:
df['A'] = df['FoC_high']/52*100
df['B'] = 100-df['A']
df['C'] = df['FoC_low']/52*100
df['D'] = 100-df['C']
df['chi'] = (df['A']*df['D']-df['B']*df['C']).pow(2)*0.02/(df['B']+df['D'])/(df['A']+df['C'])
df.loc[df['chi']>=3.841, 'p=0.05'] = 1
df = df.fillna(0)
df

In [ ]:
df.loc[(df['p=0.05']==1) & (df['A']>=50) & (df['C']>=50), 'label'] = 'Med'
df.loc[(df['p=0.05']==1) & (df['A']>=50) & (df['C']<50), 'label'] = 'FH'
df.loc[(df['p=0.05']==1) & (df['A']<50) & (df['C']>=50), 'label'] = 'FL'
df

In [ ]:
file = open('third_model_data/quiz5_high.txt', 'r')
P_high = file.readlines()
file.close()
P_high = [x[x.index(' ') + 1: -1] for x in P_high]
P_high = [x.split() for x in P_high]
file = open('third_model_data/quiz5_low.txt', 'r')
P_low = file.readlines()
file.close()
P_low = [x[x.index(' ') + 1: -1] for x in P_low]
P_low = [x.split() for x in P_low]

In [ ]:
def FoC(df, FP, P_high, P_low):
    #is = instance support
    list_is_high = []
    list_is_low = []
    for item in P_high:
        item_flat = ' '.join(item)
        x = re.findall(FP, item_flat)
        list_is_high.append(len(x))
    for item in P_low:
        item_flat = ' '.join(item)
        x = re.findall(FP, item_flat)
        list_is_low.append(len(x))
    #print(list_is_high)
    #print('*'*50)
    #print(list_is_low)
    return ttest_ind(list_is_high, list_is_low)[1], sum(list_is_high), sum(list_is_low)

In [ ]:
df_subset = df.loc[(df['label']=='Med')]
for i in df_subset.index:
    aaa = FoC(df, df.loc[i, 'FP'], P_high, P_low)
    if aaa[0] >= 0.05:
        print(aaa)
    else:
        print(aaa)
        if aaa[1] > aaa[2]:
            df.loc[i, 'label'] = 'DH'
        else:
            df.loc[i, 'label'] = 'DL'

In [ ]:
df.loc[(df['label']=='FH') | (df['label']=='FL')| (df['label']=='DH')| (df['label']=='DL')]

In [ ]:
df_student_feature = df.loc[(df['label']=='FH') | (df['label']=='FL')| (df['label']=='DH')| (df['label']=='DL')].T.head(1)
df_student_feature

In [ ]:
for i in range(len(P_high)):
    student_flat = ' '.join(P_high[i])
    df_student_feature.loc[len(df_student_feature.index)] = [student_flat] * len(df_student_feature.columns)
for i in range(len(P_low)):
    student_flat = ' '.join(P_low[i])
    df_student_feature.loc[len(df_student_feature.index)] = [student_flat] * len(df_student_feature.columns)
df_student_feature

In [ ]:
for i in range(len(df_student_feature.columns)):
    df_student_feature.iloc[:,i] = df_student_feature.iloc[:,i].str.count(df_student_feature.iloc[0,i])
df_student_feature = df_student_feature.iloc[1:]
#df_student_feature[df_student_feature>1] = 1
df_student_feature

In [ ]:
df_student_feature.to_csv('third_model_data/features/quiz5.csv', index =False)

In [2]:
df_student_feature1 = pd.read_csv('third_model_data/features/quiz1.csv')
df_student_feature1.index = np.arange(1, len(df_student_feature1)+1)
df_student_feature2 = pd.read_csv('third_model_data/features/quiz2.csv')
df_student_feature2.index = np.arange(1, len(df_student_feature2)+1)
df_student_feature3 = pd.read_csv('third_model_data/features/quiz3.csv')
df_student_feature3.index = np.arange(1, len(df_student_feature3)+1)
df_student_feature4 = pd.read_csv('third_model_data/features/quiz4.csv')
df_student_feature4.index = np.arange(1, len(df_student_feature4)+1)

In [19]:
df_student_feature = pd.merge(df_student_feature1, df_student_feature2, left_index=True, right_index=True)
df_student_feature = pd.merge(df_student_feature, df_student_feature3, left_index=True, right_index=True)
df_student_feature = pd.merge(df_student_feature, df_student_feature4, left_index=True, right_index=True)
#df_student_feature = df_student_feature1

In [20]:
X = df_student_feature.to_numpy()
X.shape

(104, 164)

In [12]:
y = np.concatenate((np.ones((77,), dtype=int), np.zeros((27,), dtype=int)), axis = None)

In [13]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.gaussian_process.kernels import RBF

In [22]:
names = [
    "Nearest Neighbors",
    "Linear SVM",
    "RBF SVM",
    "Gaussian Process",
    "Decision Tree",
    "Random Forest",
    "Neural Net",
    "AdaBoost",
    "Naive Bayes",
    "QDA",
]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis(),
]

In [23]:
for name, clf in zip(names, classifiers):
    clf.fit(X, y)
    scores = cross_val_score(clf, X, y, cv=5)
    print(name, scores.mean())

Nearest Neighbors 0.6247619047619047
Linear SVM 0.7109523809523809
RBF SVM 0.7404761904761905
Gaussian Process 0.6638095238095237
Decision Tree 0.6066666666666667
Random Forest 0.7119047619047619
Neural Net 0.6042857142857143
AdaBoost 0.5466666666666666
Naive Bayes 0.6242857142857142
QDA 0.5676190476190477


C:\Coding\Anaconda3\envs\cisc320\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Coding\Anaconda3\envs\cisc320\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Coding\Anaconda3\envs\cisc320\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Coding\Anaconda3\envs\cisc320\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Coding\Anaconda3\envs\cisc320\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Coding\Anaconda3\envs\cisc320\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables ar